# Preprocessing dataset

**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

**Importing dataset**

In [2]:
dataset = pd.read_csv("data/bank-additional-full.csv", sep=";")

## Features exploration and description

## Checking missing values

In [3]:
dataset.head(pd.set_option("display.max_columns", None))

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [4]:
dataset.isnull().values.any()

False

## Binning age feature

In [5]:
dataset['age'].describe()

count    41188.00000
mean        40.02406
std         10.42125
min         17.00000
25%         32.00000
50%         38.00000
75%         47.00000
max         98.00000
Name: age, dtype: float64

In [6]:
dataset['age_groups'] = pd.cut(dataset['age'],
                            [10,20,30,40,50,60,70,80,90,100])

In [7]:
dataset['age_groups']

0        (50, 60]
1        (50, 60]
2        (30, 40]
3        (30, 40]
4        (50, 60]
           ...   
41183    (70, 80]
41184    (40, 50]
41185    (50, 60]
41186    (40, 50]
41187    (70, 80]
Name: age_groups, Length: 41188, dtype: category
Categories (9, interval[int64, right]): [(10, 20] < (20, 30] < (30, 40] < (40, 50] ... (60, 70] < (70, 80] < (80, 90] < (90, 100]]

## Modifying education feature

In [8]:
dataset['education'].unique()
dataset['education'] = dataset['education'].replace(
    to_replace=['basic.4y','basic.6y','basic.9y'], value = 'basic')
dataset['education'].unique()

array(['basic', 'high.school', 'professional.course', 'unknown',
       'university.degree', 'illiterate'], dtype=object)

## Dropping duration feature

In [9]:
dataset.drop('duration', axis=1, inplace=True)

## Binarizing pdays feature

In [10]:
dataset['pdays'].unique()

array([999,   6,   4,   3,   5,   1,   0,  10,   7,   8,   9,  11,   2,
        12,  13,  14,  15,  16,  21,  17,  18,  22,  25,  26,  19,  27,
        20])

Most of the unique values for this feature are less than 30, while the vast majority of the values are 999 because those customers (records) were not contacted previously from a previous campaign. Thus, we can binarize this feature to simplify it. 0 if the value is 999 (customer never contacted), 1 otherwise.

In [11]:
dataset['pdays'] = np.where(dataset['pdays'] != 999, 1, 0)
dataset['pdays'].unique()

array([0, 1])

## Class Label Balance

In [12]:
dataset['y'].value_counts() / len(dataset['y'])

no     0.887346
yes    0.112654
Name: y, dtype: float64

We can see that the class labels are highly imbalanced with 88.74% no's and only 11.23% yes'. We can use SMOTE when we fit the models in order to create synethetic records to balance the dataset.

## Recoding class labels from 'yes' and 'no' strings to 1 and 0 integers

In [13]:
dataset['y'] = dataset['y'].map({'no': 0, 'yes': 1}).astype(int)
dataset['y']

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, Length: 41188, dtype: int64

## One-hot encoding for categorical variables

In [14]:
cats = ['job', 'marital', 'education', 'default', 'housing', 
        'loan', 'contact', 'month', 'day_of_week', 'poutcome',
        'age_groups']

dataset = pd.get_dummies(dataset, 
                             columns = cats, 
                             drop_first = True)
dataset

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,marital_unknown,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,"age_groups_(20, 30]","age_groups_(30, 40]","age_groups_(40, 50]","age_groups_(50, 60]","age_groups_(60, 70]","age_groups_(70, 80]","age_groups_(80, 90]","age_groups_(90, 100]"
0,56,1,0,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,57,1,0,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
2,37,1,0,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
3,40,1,0,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
4,56,1,0,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,0,0,-1.1,94.767,-50.8,1.028,4963.6,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
41184,46,1,0,0,-1.1,94.767,-50.8,1.028,4963.6,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
41185,56,2,0,0,-1.1,94.767,-50.8,1.028,4963.6,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
41186,44,1,0,0,-1.1,94.767,-50.8,1.028,4963.6,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
